## 日付データやその他情報を追加

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm

 データの読み込み

In [2]:
pd.set_option('display.max_rows',5)
pd.options.display.width = 500
pd.options.display.column_space = 200

df = pd.read_pickle('df.pickle')
race_id_list = df.index.unique()
str_id_list = list(map(str,race_id_list))
str_id_list

['202103010101',
 '202103010102',
 '202103010103',
 '202103010104',
 '202103010105',
 '202103010106',
 '202103010107',
 '202103010108',
 '202103010109',
 '202103010110',
 '202103010111',
 '202103010112',
 '202102010101',
 '202102010102',
 '202102010103',
 '202102010104',
 '202102010105',
 '202102010106',
 '202102010107',
 '202102010108',
 '202102010109',
 '202102010110',
 '202102010111',
 '202102010112']

 ---

In [3]:
race_infos = {}
def add_data(race_id_list):
  for race_id in tqdm(race_id_list):
    url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
    html = requests.get(url)
    html.encoding= "EUC_JP"
    soup = BeautifulSoup(html.text, "html.parser")
    time.sleep(1)
  
    日付 = soup.find("dd",class_='Active')
    日付 = 日付.find("a").get_text()
    レース情報 = soup.find("div", class_="RaceList_Item02") 
    # 距離 = レース情報.find("div",class_="RaceData01").find("span").get_text()
    # 馬場 = レース情報.find("div",class_="RaceData01").find("span",class_="Item03").get_text()
    レース環境 = レース情報.find("div",class_="RaceData01").get_text()
    info = re.findall(r"\w+",レース環境)
    info[0] = info[2][:1]
    info[1] = info[2][1:]
    info.pop(2)
    info_dict = {}
    # print(info)
    info_dict["date"] = f"{race_id[:4]}年{日付[:-3]}"

    for text in info:
      # print(text)
      if text in ["芝","ダ"]:
        # print("芝付き")
        info_dict["race_type"] = text
      if "障" in text:
        info_dict["race_type"] = "障害"
      if "m" in text:
        # print("mつき")
        info_dict["course_ren"] = re.findall(r"\d+",text)[0]
      if text in ["雨","小雨","小雪","雪","晴","曇"]:
        # print("天気")
        info_dict["weather"] = text
      # if "年" in text:
      #   info_dict["data"] = text
      if text in ["良","不良","重","稍","不"]:
        # print("馬場")
        info_dict["ground_state"] = text


    race_infos[race_id] = info_dict
  return race_infos

In [4]:
results = add_data(str_id_list)

100%|██████████| 24/24 [00:32<00:00,  1.37s/it]


----

## dataの形を変形

In [5]:
race_infos_df = pd.DataFrame(results).T

結合するdataframeの形が一致しないと結合できないので注意

In [6]:
df.index = df.index.astype(str)

In [7]:
race_infos_df.index = race_infos_df.index.astype(str)

strに変わったか確認

In [8]:
type(df.index[0])

str

In [9]:
pd.set_option('display.max_rows',None)

inner_df = df.merge(race_infos_df,left_index=True,right_index=True,how="inner")
# inner_df = pd.merge(df, race_infos_df, left_index=True,right_index=True, how="inner")
# print(type(inner_df))
# print(len(inner_df))
# inner_df

----

In [10]:
inner_df.to_pickle("inner_df.pickle")